In [1]:
from pykrx import stock
import pandas as pd
from time import sleep
import re

In [3]:
stock_list = pd.read_csv("../resources/stockcode.csv", dtype = {"종목코드": str, "회사명": str})

In [25]:
first_df = pd.read_csv('../resources/ohlcv_p1p2p3_nasdq/000020.csv', parse_dates=['date'], index_col=[0])

,date,open,high,low,close,volume,pattern1,pattern2,pattern3,nasdaq
0,2012-01-02,4070,4100,4060,4085,25451,K00,NaN,NaN,NaN
1,2012-01-03,4100,4160,4100,4130,45291,P03,K00P03,NaN,NaN
2,2012-01-04,4135,4160,4125,4150,42911,K00,P00K00,P10K00,U04
3,2012-01-05,4150,4290,4110,4270,85140,P05,K00P05,P10P05,T01
4,2012-01-06,4300,4325,4220,4240,45535,M05,P00M05,P10M05,U04
...,...,...,...,...,...,...,...,...,...,...
2165,2020-10-21,20200,20500,19850,20100,402212,M05,M00M05,M10M05,U04
2166,2020-10-22,19850,20050,19000,19200,620779,M05,M00M05,M10M05,D04
2167,2020-10-23,19500,20600,19350,19800,563966,P05,M00P05,M10P05,U03
2168,2020-10-26,19500,19750,17000,17100,1338181,M15,P00M15,S01M15,U04


In [29]:
pd_last_date = first_df['date'].iloc[-1]

Timestamp('2020-10-27 00:00:00')

In [31]:
pd_today = pd.to_datetime(pd.Timestamp.now().date())

Timestamp('2020-10-27 00:00:00')

In [32]:
# 데이터를 갱신할 날짜 범위
pd_last_p1 = pd_last_date + pd.Timedelta(days=1)
pd_drange = pd.date_range(pd_last_p1, pd_today)
if pd_last_p1 == pd_today:
    pd_drange = []

In [71]:
pd_date = pd_today

In [ ]:
# for pd_date in pd_drange:
#     df_update = stock.get_market_ohlcv_by_ticker(pd_date)
#     if not len(df_update):  # 장이 열리지 않은 날이면 skip
#         continue
#     df_update = df_update.reset_index()
#     for stock_ in df_update.iloc:
#         stock_code = stock_['종목코드']
#         filename= f'resources/ohlcv/{stock_code}.csv'
#         str_date = str(pd_date.date())
#         if (isfile(filename)):
#             with open(filename,'a',encoding='UTF-8') as f:
#                 f.write(f"{str_date},{stock_['시가']},{stock_['고가']},{stock_['저가']},{stock_['종가']},{stock_['거래량']}\n")
#         else:
#             new_stock = stock.get_market_ohlcv_by_date("20120101", pd_date, stock_code)
#             new_stock.to_csv(filename, encoding='UTF-8')
#             sleep(1)

In [73]:
df_update = stock.get_market_ohlcv_by_ticker(pd_today)
df_update = df_update.reset_index()

In [74]:
stock_ = df_update.iloc[0]

In [75]:
stock_code = stock_['종목코드']

In [79]:
try:
    update_target = pd.read_csv(f'../resources/ohlcv/{stock_code}.csv', parse_dates=['날짜'])
except:
    pass
update_target = update_target.iloc[:2169]

In [81]:
update_target = update_target.append({'날짜':pd_date,'시가':stock_['시가'],'고가':stock_['고가'],'저가':stock_['저가'],'종가':stock_['종가'],'거래량':stock_['거래량']}, 
                                     ignore_index=True)
update_target

,날짜,시가,고가,저가,종가,거래량
0,2012-01-02,4070,4100,4060,4085,25451
1,2012-01-03,4100,4160,4100,4130,45291
2,2012-01-04,4135,4160,4125,4150,42911
3,2012-01-05,4150,4290,4110,4270,85140
4,2012-01-06,4300,4325,4220,4240,45535
...,...,...,...,...,...,...
2165,2020-10-21,20200,20500,19850,20100,402212
2166,2020-10-22,19850,20050,19000,19200,620779
2167,2020-10-23,19500,20600,19350,19800,563966
2168,2020-10-26,19500,19750,17000,17100,1338181


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2169 entries, 0 to 2168
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      2169 non-null   datetime64[ns]
 1   시가      2169 non-null   int64         
 2   고가      2169 non-null   int64         
 3   저가      2169 non-null   int64         
 4   종가      2169 non-null   int64         
 5   거래량     2169 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 101.8 KB
